In [520]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

from hyperopt import fmin, hp, tpe, Trials
from scipy.io import mmwrite

In [521]:
size_cloth_column = ['newClothId', 'clothMale', 'clothFemale', 'clothRate', 'clothReviewCnt', 'userHeight', 'userWeight', 'userMale', 'userFemale']
size_user_column = ['userId','userMale', 'userFemale', 'userHeight', 'userWeight', 'clothCnt']
favor_ctop_column = ['newClothId', 'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
        'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
        'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'userMale', 'userFemale', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_couter_column = ['newClothId', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
        'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_cpants_column = ['newClothId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_conepiece_column = ['newClothId', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_cskirt_column = ['newClothId', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_utop_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'touch', 'smallCategoryHalfshort', 'smallCategoryShirt',
       'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit', 'smallCategoryLong', 'smallCategoryShort',
       'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12', 'clothCnt']
favor_uouter_column = ['userId', 'size', 'bright', 'color', 'thickness',
       'weightness', 'touch', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12', 'clothCnt']
favor_upants_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12', 'clothCnt']
favor_uonepiece_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12', 'clothCnt']
favor_uskirt_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12', 'clothCnt']

In [523]:
transaction = pd.read_json('./transaction_skirt.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]
transactiontransaction = pd.read_json('./transaction_skirt.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]
transaction

,userId,newClothId,shopCnt
0,0,19641661812,1
1,1,1795772837,1
2,1,21172562649,1
3,2,20995251016,1
4,3,19136511221,1
...,...,...,...
10306,8292,15601831248,2
10307,8293,2214923407,1
10308,8294,20462111605,1
10309,8295,21014592225,1


In [524]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))
a

,newClothId,idx
0,19641661812,0
1,1795772837,1
2,21172562649,2
3,20995251016,3
4,19136511221,4
...,...,...
10274,1773210461,2649
10275,20704491449,2650
10280,21313542109,2651
10281,233981145,2652


In [632]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")
transaction

,userId,newClothId,shopCnt,idx
0,0,19641661812,1,0
1,1,1795772837,1,1
2,1,21172562649,1,2
3,2,20995251016,1,3
4,3,19136511221,1,4
...,...,...,...,...
10306,8292,15601831248,2,17
10307,8293,2214923407,1,38
10308,8294,20462111605,1,376
10309,8295,21014592225,1,1458


In [526]:
user_meta = pd.read_json('./user_skirt.json')
cloth_meta = pd.read_json('./final_skirt.json')

In [527]:
user_meta = user_meta[size_user_column]
user_meta

,userId,userMale,userFemale,userHeight,userWeight,clothCnt
0,0,0,1,160,55,1
1,1,0,1,160,49,2
2,2,0,1,155,50,1
3,3,0,1,161,44,1
4,4,0,1,164,47,4
...,...,...,...,...,...,...
8292,8292,0,1,157,55,2
8293,8293,0,1,152,45,1
8294,8294,0,1,169,48,1
8295,8295,0,1,160,48,1


In [784]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('userMale')] = 0
new_user.loc[new_user_idx, ('userFemale')] = 1
new_user.loc[new_user_idx, ('userHeight')] = 172
new_user.loc[new_user_idx, ('userWeight')] = 60
new_user.loc[new_user_idx, ('clothCnt')] = 0
new_user

,userId,userMale,userFemale,userHeight,userWeight,clothCnt
0,8299,0,1,172,60,0


In [790]:
new_user.userFemale

0    1
Name: userFemale, dtype: int64

In [792]:
sub_user = user_meta[user_meta.userFemale==new_user.userFemale.iloc[0]]

In [793]:
new_user_height = new_user.userHeight.iloc[0]
new_user_weight = new_user.userWeight.iloc[0]
user_list = []
for info in sub_user.itertuples():
    if info.userHeight == new_user_height and info.userWeight == new_user_weight:
        user_list.append(info.userId)

In [805]:
import random
real_user = random.choice(user_list)
real_user

5892

In [785]:
# 사이즈
# 여성 남성은 확실하게 나눠서 비교
# 각 파트에서 userHeight, userWeight가 같은 사람이 있을 것!
# 그걸 조회하는 별로 안걸림
# random으로 1개를 선택

# 취향
# 이게 문제

In [528]:
cloth_meta = cloth_meta[size_cloth_column]
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale
0,2018790,0,1,0.0,0,0.0,0,0.0,1.0
1,1844574,0,1,0.0,0,0.0,0,0.0,1.0
2,1672093,0,1,0.0,0,0.0,0,0.0,1.0
3,1687533,0,1,0.0,0,0.0,0,0.0,1.0
4,2338446,0,1,0.0,0,0.0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
8871,21172562649,0,1,4.8,148,165.0,55,0.0,1.0
8872,21172562648,0,1,4.8,148,175.0,65,0.0,1.0
8873,23400702652,0,1,4.7,14,160.0,55,0.0,1.0
8874,23400702653,0,1,4.7,14,161.0,57,0.0,1.0


In [529]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
0,2018790,0,1,0.0,0,0.0,0,0.0,1.0,NaN
1,1844574,0,1,0.0,0,0.0,0,0.0,1.0,NaN
2,1672093,0,1,0.0,0,0.0,0,0.0,1.0,NaN
3,1687533,0,1,0.0,0,0.0,0,0.0,1.0,NaN
4,2338446,0,1,0.0,0,0.0,0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
8871,21172562649,0,1,4.8,148,165.0,55,0.0,1.0,2.0
8872,21172562648,0,1,4.8,148,175.0,65,0.0,1.0,531.0
8873,23400702652,0,1,4.7,14,160.0,55,0.0,1.0,1441.0
8874,23400702653,0,1,4.7,14,161.0,57,0.0,1.0,1273.0


In [530]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2654

In [531]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
0,161812522,0,1,4.6,10,165.000000,54,0.0,1.0,1682.0
1,161812521,0,1,4.6,10,162.000000,60,0.0,1.0,896.0
2,185310349,0,1,5.0,1,167.333333,57,0.0,1.0,1662.0
3,1884867110,0,1,5.0,2,167.000000,56,0.0,1.0,2304.0
4,1629303146,0,1,4.8,16,162.000000,44,0.0,1.0,965.0
...,...,...,...,...,...,...,...,...,...,...
2649,21172562650,0,1,4.8,148,161.000000,46,0.0,1.0,22.0
2650,21172562649,0,1,4.8,148,165.000000,55,0.0,1.0,2.0
2651,21172562648,0,1,4.8,148,175.000000,65,0.0,1.0,531.0
2652,23400702652,0,1,4.7,14,160.000000,55,0.0,1.0,1441.0


In [532]:
cloth_meta.idx = cloth_meta.idx.astype('int')
cloth_meta

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
0,161812522,0,1,4.6,10,165.000000,54,0.0,1.0,1682
1,161812521,0,1,4.6,10,162.000000,60,0.0,1.0,896
2,185310349,0,1,5.0,1,167.333333,57,0.0,1.0,1662
3,1884867110,0,1,5.0,2,167.000000,56,0.0,1.0,2304
4,1629303146,0,1,4.8,16,162.000000,44,0.0,1.0,965
...,...,...,...,...,...,...,...,...,...,...
2649,21172562650,0,1,4.8,148,161.000000,46,0.0,1.0,22
2650,21172562649,0,1,4.8,148,165.000000,55,0.0,1.0,2
2651,21172562648,0,1,4.8,148,175.000000,65,0.0,1.0,531
2652,23400702652,0,1,4.7,14,160.000000,55,0.0,1.0,1441


In [533]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]
transaction_source

[(0, 0),
 (1, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (5, 9),
 (6, 10),
 (7, 11),
 (8, 12),
 (9, 13),
 (10, 14),
 (11, 15),
 (12, 16),
 (13, 17),
 (13, 18),
 (14, 19),
 (15, 20),
 (16, 21),
 (16, 22),
 (17, 23),
 (18, 24),
 (18, 9),
 (19, 25),
 (20, 26),
 (21, 27),
 (22, 28),
 (23, 29),
 (24, 30),
 (25, 31),
 (26, 32),
 (26, 33),
 (27, 34),
 (28, 35),
 (29, 36),
 (30, 37),
 (31, 38),
 (32, 39),
 (33, 17),
 (34, 40),
 (34, 2),
 (35, 41),
 (36, 42),
 (37, 43),
 (38, 44),
 (39, 45),
 (40, 46),
 (41, 47),
 (42, 48),
 (43, 49),
 (44, 50),
 (45, 51),
 (46, 52),
 (47, 53),
 (47, 54),
 (48, 55),
 (49, 56),
 (50, 57),
 (51, 58),
 (52, 59),
 (53, 22),
 (53, 60),
 (54, 61),
 (55, 62),
 (56, 63),
 (57, 64),
 (58, 65),
 (59, 66),
 (60, 67),
 (61, 68),
 (62, 69),
 (63, 70),
 (63, 71),
 (64, 72),
 (64, 37),
 (64, 73),
 (65, 74),
 (66, 75),
 (67, 76),
 (68, 77),
 (68, 78),
 (69, 79),
 (69, 80),
 (70, 81),
 (70, 82),
 (70, 83),
 (71, 84),
 (72, 0),
 (73, 85),
 (73, 86),
 (74

In [534]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.newClothId.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 8297
clothId의 유니크 개수 : 2654
user테이블의 userId의 유니크 개수 : 8297
cloth테이블의 clothId의 유니크 개수 : 2654


In [535]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i], user_meta['clothCnt'][i]]) for i in range(user_meta.shape[0])]
user_features_source

[(0, [0, 1, 160, 55, 1]),
 (1, [0, 1, 160, 49, 2]),
 (2, [0, 1, 155, 50, 1]),
 (3, [0, 1, 161, 44, 1]),
 (4, [0, 1, 164, 47, 4]),
 (5, [0, 1, 162, 57, 2]),
 (6, [0, 1, 163, 49, 1]),
 (7, [0, 1, 161, 45, 1]),
 (8, [0, 1, 166, 53, 1]),
 (9, [0, 1, 158, 44, 1]),
 (10, [0, 1, 162, 60, 1]),
 (11, [0, 1, 162, 53, 1]),
 (12, [0, 1, 158, 60, 1]),
 (13, [0, 1, 162, 52, 3]),
 (14, [0, 1, 156, 42, 1]),
 (15, [0, 1, 161, 42, 1]),
 (16, [0, 1, 170, 54, 4]),
 (17, [0, 1, 163, 57, 1]),
 (18, [0, 1, 165, 47, 2]),
 (19, [0, 1, 169, 58, 1]),
 (20, [0, 1, 164, 60, 2]),
 (21, [0, 1, 168, 60, 1]),
 (22, [0, 1, 163, 50, 1]),
 (23, [0, 1, 150, 38, 1]),
 (24, [0, 1, 165, 50, 1]),
 (25, [0, 1, 165, 55, 1]),
 (26, [0, 1, 170, 68, 2]),
 (27, [0, 1, 168, 57, 1]),
 (28, [0, 1, 158, 44, 2]),
 (29, [0, 1, 160, 50, 1]),
 (30, [0, 1, 160, 50, 1]),
 (31, [0, 1, 163, 52, 1]),
 (32, [0, 1, 169, 49, 1]),
 (33, [0, 1, 172, 52, 1]),
 (34, [0, 1, 160, 48, 2]),
 (35, [0, 1, 164, 47, 1]),
 (36, [0, 1, 167, 60, 1]),
 (37, [0, 1

In [536]:
# cloth_features_source = [(cloth_meta['idx'][i],
#                         [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i], cloth_meta['clothRate'][i], cloth_meta['clothReviewCnt'][i],
#                          cloth_meta['userHeight'][i], cloth_meta['userWeight'][i], cloth_meta['userMale'][i], cloth_meta['userFemale'][i], ]) for i in range(cloth_meta.shape[0])]
# cloth_features_source

[(1682, [0, 1, 4.6, 10, 165.0, 54, 0.0, 1.0]),
 (896, [0, 1, 4.6, 10, 162.0, 60, 0.0, 1.0]),
 (1662, [0, 1, 5.0, 1, 167.3333333333, 57, 0.0, 1.0]),
 (2304, [0, 1, 5.0, 2, 167.0, 56, 0.0, 1.0]),
 (965, [0, 1, 4.8, 16, 162.0, 44, 0.0, 1.0]),
 (904, [0, 1, 5.0, 1, 164.0, 51, 0.0, 1.0]),
 (551, [0, 1, 5.0, 9, 172.0, 56, 0.0, 1.0]),
 (1590, [0, 1, 4.7, 7, 158.0, 65, 0.0, 1.0]),
 (2457, [0, 1, 4.7, 7, 165.0, 52, 0.0, 1.0]),
 (1354, [0, 1, 5.0, 2, 157.5, 44, 0.0, 1.0]),
 (1881, [0, 1, 4.3, 22, 158.5, 54, 0.0, 1.0]),
 (96, [0, 1, 4.5, 2, 161.8, 49, 0.0545454545, 0.9454545455000001]),
 (653, [0, 1, 4.9, 8, 160.0, 54, 0.0, 1.0]),
 (1920, [0, 1, 4.9, 8, 161.25, 52, 0.0, 1.0]),
 (2483, [0, 1, 5.0, 2, 168.0, 56, 0.0, 1.0]),
 (1625, [0, 1, 4.6, 8, 168.0, 55, 0.0, 1.0]),
 (2591, [0, 1, 4.6, 8, 160.0, 48, 0.0, 1.0]),
 (1982, [0, 1, 4.6, 8, 158.5, 46, 0.0, 1.0]),
 (305, [0, 1, 4.5, 15, 160.0, 48, 0.0, 1.0]),
 (2223, [0, 1, 5.0, 2, 163.3529411765, 57, 0.0, 1.0]),
 (2208, [0, 1, 4.8, 20, 164.0, 65, 0.0, 

In [662]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]
cloth_features_source

[(1682, [0, 1, 165.0, 54]),
 (896, [0, 1, 162.0, 60]),
 (1662, [0, 1, 167.3333333333, 57]),
 (2304, [0, 1, 167.0, 56]),
 (965, [0, 1, 162.0, 44]),
 (904, [0, 1, 164.0, 51]),
 (551, [0, 1, 172.0, 56]),
 (1590, [0, 1, 158.0, 65]),
 (2457, [0, 1, 165.0, 52]),
 (1354, [0, 1, 157.5, 44]),
 (1881, [0, 1, 158.5, 54]),
 (96, [0, 1, 161.8, 49]),
 (653, [0, 1, 160.0, 54]),
 (1920, [0, 1, 161.25, 52]),
 (2483, [0, 1, 168.0, 56]),
 (1625, [0, 1, 168.0, 55]),
 (2591, [0, 1, 160.0, 48]),
 (1982, [0, 1, 158.5, 46]),
 (305, [0, 1, 160.0, 48]),
 (2223, [0, 1, 163.3529411765, 57]),
 (2208, [0, 1, 164.0, 65]),
 (768, [0, 1, 158.0, 49]),
 (5, [0, 1, 163.6666666667, 46]),
 (1305, [0, 1, 158.6666666667, 52]),
 (1558, [0, 1, 159.0, 50]),
 (1479, [0, 1, 153.0, 37]),
 (1278, [0, 1, 160.0, 50]),
 (1780, [0, 1, 167.0, 58]),
 (2097, [0, 1, 159.0, 52]),
 (280, [0, 1, 160.0, 50]),
 (2512, [0, 1, 163.0, 47]),
 (1607, [0, 1, 153.0, 42]),
 (2247, [0, 1, 166.0, 50]),
 (2410, [0, 1, 164.6363636364, 55]),
 (2193, [0, 1, 

In [663]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [664]:
interactions, weights = dataset.build_interactions(transaction_source)

In [665]:
transaction_source

[(0, 0),
 (1, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (5, 9),
 (6, 10),
 (7, 11),
 (8, 12),
 (9, 13),
 (10, 14),
 (11, 15),
 (12, 16),
 (13, 17),
 (13, 18),
 (14, 19),
 (15, 20),
 (16, 21),
 (16, 22),
 (17, 23),
 (18, 24),
 (18, 9),
 (19, 25),
 (20, 26),
 (21, 27),
 (22, 28),
 (23, 29),
 (24, 30),
 (25, 31),
 (26, 32),
 (26, 33),
 (27, 34),
 (28, 35),
 (29, 36),
 (30, 37),
 (31, 38),
 (32, 39),
 (33, 17),
 (34, 40),
 (34, 2),
 (35, 41),
 (36, 42),
 (37, 43),
 (38, 44),
 (39, 45),
 (40, 46),
 (41, 47),
 (42, 48),
 (43, 49),
 (44, 50),
 (45, 51),
 (46, 52),
 (47, 53),
 (47, 54),
 (48, 55),
 (49, 56),
 (50, 57),
 (51, 58),
 (52, 59),
 (53, 22),
 (53, 60),
 (54, 61),
 (55, 62),
 (56, 63),
 (57, 64),
 (58, 65),
 (59, 66),
 (60, 67),
 (61, 68),
 (62, 69),
 (63, 70),
 (63, 71),
 (64, 72),
 (64, 37),
 (64, 73),
 (65, 74),
 (66, 75),
 (67, 76),
 (68, 77),
 (68, 78),
 (69, 79),
 (69, 80),
 (70, 81),
 (70, 82),
 (70, 83),
 (71, 84),
 (72, 0),
 (73, 85),
 (73, 86),
 (74

In [666]:
interactions.todense()[4]

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [667]:
interactions

<8297x2654 sparse matrix of type '<class 'numpy.int32'>'
	with 10311 stored elements in COOrdinate format>

In [668]:
weights

<8297x2654 sparse matrix of type '<class 'numpy.float32'>'
	with 10311 stored elements in COOrdinate format>

In [669]:
user_features = dataset.build_user_features(user_features_source)
user_features

<8297x8297 sparse matrix of type '<class 'numpy.float32'>'
	with 44181 stored elements in Compressed Sparse Row format>

In [670]:
cloth_features = dataset.build_item_features(cloth_features_source)
cloth_features

<2654x3181 sparse matrix of type '<class 'numpy.float32'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [671]:
train, test = random_train_test_split(interactions, test_percentage=0.1)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [672]:
train

<8297x2654 sparse matrix of type '<class 'numpy.intc'>'
	with 9279 stored elements in COOrdinate format>

In [673]:
test

<8297x2654 sparse matrix of type '<class 'numpy.intc'>'
	with 1032 stored elements in COOrdinate format>

In [674]:
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 10)),
         hp.uniform('learning_rate', 0.01, 0.05)]

In [675]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=10,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [676]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 10, lrn_rate: 0.02316, precision: 0.01079    
no_comp: 30, lrn_rate: 0.01679, precision: 0.01079                                 
no_comp: 40, lrn_rate: 0.04974, precision: 0.01119                                 
no_comp: 10, lrn_rate: 0.04568, precision: 0.01099                                 
no_comp: 20, lrn_rate: 0.02998, precision: 0.01059                                
no_comp: 20, lrn_rate: 0.03044, precision: 0.01239                                
no_comp: 30, lrn_rate: 0.02521, precision: 0.01059                                 
no_comp: 30, lrn_rate: 0.01086, precision: 0.00919                                 
no_comp: 20, lrn_rate: 0.04226, precision: 0.01079                                 
no_comp: 20, lrn_rate: 0.03656, precision: 0.01099                                 
100%|██████████| 10/10 [00:19<00:00,  1.97s/trial, best loss: -0.012387611903250217]


In [677]:
best_params

{'learning_rate': 0.030441746065469635, 'no_components': 1}

In [678]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.03 1


In [ ]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=train,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=train_weights,
          epochs=10,
          verbose=False)

In [737]:
# 모델 저장하기
import pickle
with open('model_size_skirt.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [738]:
# 모델 불러오기
import pickle
with open('model_size_skirt.pickle', 'rb') as f:
    model_skirt = pickle.load(f)

In [773]:
rec_size_skirt = sample_recommendation(model_skirt, cloth_meta.idx, 5120)
rec_size_skirt

[2017517424,
 1618698368,
 1775973474,
 1907111403,
 1926066515,
 2125714273,
 150926840,
 2054486283,
 1173636175,
 180781478,
 117363976,
 1618698369,
 1969751297,
 117361628,
 1618698367,
 2205663410,
 150926841,
 144327224,
 1075602105,
 1969751296,
 130965253,
 1834795393,
 2060320310,
 1645762423,
 1283437234,
 1173636176,
 1248001459,
 2236800983,
 1917890219,
 1865575569]

model.fit(interactions=test,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=test_weights,
          epochs=10,
          verbose=False)

In [772]:
def sample_recommendation(model, idx, user_id):
    scores = model.predict(user_id, np.arange(len(cloth_meta)))

    top_items = idx[np.argsort(-scores)]

    sub_clothes = []

    for x in top_items[:30]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [741]:
n_items = train.shape[1]
np.arange(len(cloth_meta))

array([   0,    1,    2, ..., 2651, 2652, 2653])

In [734]:
user_meta[user_meta.userId==5120]

,userId,userMale,userFemale,userHeight,userWeight,clothCnt
5120,5120,0,1,163,48,1


In [733]:
cloth_meta[cloth_meta.idx==85]

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale,idx
38,1618698368,0,1,4.7,400,164.0,52,0.0,1.0,85


In [731]:
sample_recommendation(model, test, cloth_meta.idx, 5120)

User 5120
     Recommended:
        305
        85
        757
        1982
        785
        1352
        124
        2171
        442
        144


In [447]:
model.fit(interactions=test,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=test_weights,
          epochs=10,
          verbose=False)

In [474]:
type(test)

scipy.sparse.coo.coo_matrix

In [477]:
type(test_case)

numpy.matrix

In [482]:
len(interactions.row)

10311

In [472]:
test_case = interactions.todense()[1000]
# test_case
train_case = interactions.todense()[:1000, 1001:]
train_case

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [504]:
shape = interactions.shape
shape

(8297, 2654)

In [510]:
cutoff = int((1.0 - 0.2) * len(uids))

train_idx = slice(None, cutoff)
test_idx = slice(1000, 1001)
test_idx

slice(1000, 1001, None)

In [511]:
import scipy.sparse as sp
interactions = interactions.tocoo()
test_idx = slice(1000, 1001)
test_case = sp.coo_matrix(
        (interactions.data[test_idx], (interactions.row[test_idx], interactions.col[test_idx])),
        shape=interactions.shape,
        dtype=interactions.dtype,
    )
test_case

<8297x2654 sparse matrix of type '<class 'numpy.int32'>'
	with 1 stored elements in COOrdinate format>

In [512]:
test_weights = test_case.multiply(weights).tocoo()

In [513]:
test_weights

<8297x2654 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in COOrdinate format>

In [515]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=test_case,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=test_weights,
          epochs=10,
          verbose=False)

In [453]:
from lightfm.evaluation import auc_score
p = auc_score(model, test, train, user_features=user_features, item_features=cloth_features)
print(len(p))

1944


In [516]:
item_biases, item_embeddings = model.get_item_representations(cloth_features)

In [519]:
item_embeddings.dot(item_embeddings[1000])

array([-0.00360154, -0.00728039, -0.00371262, ..., -0.00393674,
        0.00136199, -0.00057853], dtype=float32)

In [351]:
# from lightfm import get_item_representations

# Find Similar Items
item_biases, item_embeddings = model.get_item_representations(cloth_features)

def make_best_items_report(item_embeddings, num_search_items=10):
    user_id = user_meta[user_meta.userId==1000].index[0]

#     # Cosine similarity
    scores = user_embeddings.dot(user_embeddings[user_id])  # (10000, )
    user_norms = np.linalg.norm(user_embeddings, axis=1)    # (10000, )
    user_norms[user_norms == 0] = 1e-10
    scores /= user_norms
#     dot_ab = np.dot(user_embeddings[user_id], item_embeddings[user_id])
#     user_norms = np.linalg.norm(user_embeddings, axis=1)
#     item_norms = np.linalg.norm(item_embeddings, axis=1)
#     scores = dot_ab/(user_norms*item_norms)

    # best: score가 제일 높은 item의 id를 num_search_items 개 만큼 가져온다.
    print(np.argpartition(scores, -num_search_items))
    best = np.argpartition(scores, -num_search_items)[-num_search_items:]
    similar_item_id_and_scores = sorted(zip(best, scores[best] / item_norms[item_id]),
                                        key=lambda x: -x[1])

    # Report를 작성할 pandas dataframe
    best_items = pd.DataFrame()

    for similar_item_id, score in similar_item_id_and_scores:
        clothId = cloth_meta.iloc[similar_item_id].newClothId
        best_items = pd.concat([best_items, cloth_meta[cloth_meta.newClothId==clothId]])

    return best_items

ValueError: You must fit the model before trying to obtain predictions.

In [352]:
make_best_items_report(user_embeddings, item_embeddings, 1000 , 10)

[3820 8296 8294 ...  873 5575 2144]


NameError: name 'item_norms' is not defined